<a href="https://colab.research.google.com/github/vfrantc/deweather/blob/main/train_decomp_dehazing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Jun 21 16:07:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    44W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from glob import glob
import shutil

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
!git clone https://github.com/vfrantc/deweather.git

Cloning into 'deweather'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 76 (delta 15), reused 67 (delta 13), pack-reused 0
Unpacking objects: 100% (76/76), done.


In [ ]:
%cd deweather/TransWeather-dehaze/

/content/deweather/TransWeather-dehaze


In [ ]:
!cp /content/drive/MyDrive/haze/data/SOTS.zip .
!cp /content/drive/MyDrive/deweather2/reside.zip .
!unzip SOTS.zip
!unzip reside.zip 
!mkdir -p data/train/
!mkdir -p data/test/outdoor/
!cp -r hazy data/train/input 
!cp -r SOTS/outdoor/hazy data/test/outdoor/input

Se truncaron las últimas líneas 5000 del resultado de transmisión.
  inflating: hazy/1822_0.8_0.08.jpg  
  inflating: hazy/1823_0.95_0.1.jpg  
  inflating: hazy/1828_0.9_0.08.jpg  
  inflating: hazy/1832_0.95_0.06.jpg  
  inflating: hazy/1875_0.8_0.06.jpg  
  inflating: hazy/1889_1_0.2.jpg     
  inflating: hazy/1911_0.9_0.06.jpg  
  inflating: hazy/1915_0.95_0.04.jpg  
  inflating: hazy/1971_0.85_0.2.jpg  
  inflating: hazy/1976_0.9_0.04.jpg  
  inflating: hazy/1977_0.9_0.2.jpg   
  inflating: hazy/1977_1_0.08.jpg    
  inflating: hazy/1979_0.85_0.12.jpg  
  inflating: hazy/1982_0.95_0.12.jpg  
  inflating: hazy/2000_0.85_0.12.jpg  
  inflating: hazy/2018_0.85_0.2.jpg  
  inflating: hazy/2020_0.9_0.1.jpg   
  inflating: hazy/2020_1_0.2.jpg     
  inflating: hazy/2026_0.9_0.04.jpg  
  inflating: hazy/2033_0.9_0.08.jpg  
  inflating: hazy/2033_0.95_0.08.jpg  
  inflating: hazy/2043_0.95_0.04.jpg  
  inflating: hazy/2060_0.8_0.04.jpg  
  inflating: hazy/2070_0.9_0.16.jpg  
  inflating: h

In [ ]:
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

import cv2
import numpy as np
from glob import glob
from tqdm.notebook import tqdm

class DecomNet(nn.Module):
    def __init__(self, channel=64, kernel_size=3):
        super(DecomNet, self).__init__()
        # Shallow feature extraction
        self.net1_conv0 = nn.Conv2d(6, channel, kernel_size * 3,  padding=4, padding_mode='replicate')
        # Activated layers!
        self.net1_convs = nn.Sequential(nn.Conv2d(channel, channel, kernel_size, padding=1, padding_mode='replicate'),
                                        nn.ReLU(),
                                        nn.Conv2d(channel, channel, kernel_size, padding=1, padding_mode='replicate'),
                                        nn.ReLU(),
                                        nn.Conv2d(channel, channel, kernel_size, padding=1, padding_mode='replicate'),
                                        nn.ReLU(),
                                        nn.Conv2d(channel, channel, kernel_size, padding=1, padding_mode='replicate'),
                                        nn.ReLU(),
                                        nn.Conv2d(channel, channel, kernel_size, padding=1, padding_mode='replicate'),
                                        nn.ReLU())
        # Final recon layer
        self.net1_recon = nn.Conv2d(channel, 6, kernel_size, padding=1, padding_mode='replicate')

    def forward(self, input_im):
        input_img = torch.cat((input_im, input_im), dim=1)
        feats0   = self.net1_conv0(input_img)
        featss   = self.net1_convs(feats0)
        outs     = self.net1_recon(featss)
        R        = torch.sigmoid(outs[:, 0:3, :, :])
        L        = torch.sigmoid(outs[:, 3:6, :, :])
        return R, L

def get_decom(trainable=True):
  net = DecomNet().cuda()
  ckpt_dict  = torch.load('decomp.tar') # , map_location=torch.device('cpu')
  net.load_state_dict(ckpt_dict)
  for p in net.parameters():
      p.requires_grad = trainable
  return net

def decom_image(image, net):
  test_low_img   = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
  test_low_img   = np.transpose(test_low_img, (2, 0, 1))
  input_low_test = np.expand_dims(test_low_img, axis=0)
  input_low_test = Variable(torch.FloatTensor(torch.from_numpy(input_low_test))).cuda()
  R_low, I_low   = net(input_low_test)
  R_low = np.clip(np.transpose(R_low.cpu().detach().numpy().squeeze(), (1, 2, 0)), 0, 1)
  I_low = np.clip(np.transpose(I_low.cpu().detach().numpy().squeeze(), (1, 2, 0)), 0, 1)
  return R_low, I_low

def process_mask(mask):
  for fname in tqdm(list(glob(mask+'*.png')) + list(glob(mask+'*.jpg'))):
    image = cv2.imread(FNAME)
    reflectance, illumination = decom_image(dehazed_image)
    illumination = (illumination*255).astype(np.uint8)
    cv2.imwrite(illumination, fname)

In [ ]:
!cp /content/drive/MyDrive/deweather2/decomp.tar .
net = get_decom()

In [ ]:
process_mask('data/train/input/')
process_mask('data/train/gt')
process_mask('data/test/outdoor/input/')
process_mask('data/test/outdoor/gt/')

In [ ]:
def process_dir(input_dir='data/train/input/', 
                gt_in='clear/', 
                gt_out='data/train/gt', 
                out_fname='data/train/reside.txt',
                preffix='input/'):
  if not os.path.exists(gt_out):
    os.makedirs(gt_out)
  
  with open(out_fname, 'w') as f:
    for fname in tqdm(glob(os.path.join(input_dir, '*.jpg'))):
      main_name = os.path.basename(fname)
      gt_name = main_name[:4] + '.jpg'
      try:
        shutil.copyfile(os.path.join(gt_in, gt_name), os.path.join(gt_out, main_name))
      except FileNotFoundError:
        img = cv2.imread(os.path.join(gt_in, gt_name).replace('.jpg', '.png'))
        cv2.imwrite(os.path.join(gt_out, main_name), img)
      f.write(preffix + main_name + '\n')

In [ ]:
print('process train')
process_dir(input_dir='data/train/input/', gt_in='clear/', gt_out='data/train/gt', out_fname='data/train/reside.txt', preffix='input/')
print('process outdoor')
process_dir(input_dir='data/test/outdoor/input/', gt_in='SOTS/outdoor/gt/', gt_out='data/test/outdoor/gt/', out_fname='data/test/outdoor.txt', preffix='outdoor/input/')

process train


  0%|          | 0/72135 [00:00<?, ?it/s]

process outdoor


  0%|          | 0/500 [00:00<?, ?it/s]

# Train the same network for dehazing (for 10 epochs and see how it works)

In [ ]:
!mkdir ./training_log/

In [ ]:
!python3 train.py  -train_batch_size 32 -exp_name Transweather -epoch_start 0 -num_epochs 100

Seed:	19
--- Hyper-parameters for training ---
learning_rate: 0.0002
crop_size: [256, 256]
train_batch_size: 32
val_batch_size: 1
lambda_loss: 0.04
--- no weight loaded ---
Rain Drop old_val_psnr: 53.75, old_val_ssim: 0.9625
Learning rate sets to 0.0002.
Epoch: 0, Iteration: 0
Epoch: 0, Iteration: 100
Epoch: 0, Iteration: 200
Epoch: 0, Iteration: 300
Epoch: 0, Iteration: 400
Epoch: 0, Iteration: 500
Epoch: 0, Iteration: 600
Epoch: 0, Iteration: 700
Epoch: 0, Iteration: 800
Epoch: 0, Iteration: 900
Epoch: 0, Iteration: 1000
Epoch: 0, Iteration: 1100
Epoch: 0, Iteration: 1200
Epoch: 0, Iteration: 1300
Epoch: 0, Iteration: 1400
Epoch: 0, Iteration: 1500
Epoch: 0, Iteration: 1600
Epoch: 0, Iteration: 1700
Epoch: 0, Iteration: 1800
Epoch: 0, Iteration: 1900
Epoch: 0, Iteration: 2000
Epoch: 0, Iteration: 2100
Epoch: 0, Iteration: 2200
Rain Drop
(464s) Epoch [1/10], Train_PSNR:23.73, Val_PSNR:73.06, Val_SSIM:0.9994
model saved
Learning rate sets to 0.0002.
Epoch: 1, Iteration: 0
Epoch: 1, Ite

In [ ]:
!zip -r dehaze_10ep.zip ./Transweather/
!cp dehaze_10ep.zip /content/drive/MyDrive/deweather

updating: Transweather/ (stored 0%)
updating: Transweather/best (deflated 7%)
updating: Transweather/latest (deflated 7%)


In [ ]:
!python test_dehaze.py -exp_name=./Transweather 

Seed:	19
cuda
--- Testing starts! ---
0123_1_0.08.jpg
0311_0.8_0.16.jpg
0212_0.9_0.2.jpg
1044_0.85_0.12.jpg
0256_0.95_0.16.jpg
1784_0.8_0.08.jpg
1709_0.95_0.08.jpg
0082_0.85_0.16.jpg
1928_0.9_0.08.jpg
0274_0.95_0.12.jpg
1949_0.8_0.2.jpg
0205_1_0.16.jpg
0149_0.9_0.08.jpg
0406_1_0.2.jpg
1981_0.8_0.2.jpg
0166_0.8_0.16.jpg
0077_0.8_0.2.jpg
0090_0.85_0.12.jpg
0171_0.85_0.12.jpg
0160_0.9_0.2.jpg
1822_1_0.08.jpg
0371_0.9_0.08.jpg
0185_1_0.16.jpg
0001_0.8_0.2.jpg
1744_0.85_0.08.jpg
1868_1_0.08.jpg
0314_0.9_0.12.jpg
0404_0.8_0.2.jpg
0083_0.85_0.12.jpg
1012_0.95_0.08.jpg
0266_0.8_0.12.jpg
0294_1_0.2.jpg
1834_1_0.08.jpg
0239_0.8_0.2.jpg
1917_0.95_0.2.jpg
0288_0.95_0.2.jpg
0137_0.8_0.2.jpg
0286_0.8_0.08.jpg
1022_0.85_0.12.jpg
0086_0.95_0.12.jpg
1861_0.85_0.2.jpg
0225_0.8_0.2.jpg
0095_0.8_0.08.jpg
0169_0.9_0.12.jpg
1837_0.9_0.08.jpg
0209_0.85_0.08.jpg
0350_0.85_0.12.jpg
0040_1_0.08.jpg
1921_0.95_0.08.jpg
1738_1_0.2.jpg
0375_0.85_0.16.jpg
0081_0.8_0.2.jpg
0074_0.8_0.2.jpg
0282_0.85_0.16.jpg
0217_1_0

In [ ]:
!zip -r out.zip results

  adding: results/ (stored 0%)
  adding: results/outdoor/ (stored 0%)
  adding: results/outdoor/Transweather/ (stored 0%)
  adding: results/outdoor/Transweather/0123_1_0.08.jpg (deflated 0%)
  adding: results/outdoor/Transweather/0311_0.8_0.16.jpg (deflated 0%)
  adding: results/outdoor/Transweather/0212_0.9_0.2.jpg (deflated 1%)
  adding: results/outdoor/Transweather/1044_0.85_0.12.jpg (deflated 3%)
  adding: results/outdoor/Transweather/0256_0.95_0.16.jpg (deflated 1%)
  adding: results/outdoor/Transweather/1784_0.8_0.08.jpg (deflated 1%)
  adding: results/outdoor/Transweather/1709_0.95_0.08.jpg (deflated 0%)
  adding: results/outdoor/Transweather/0082_0.85_0.16.jpg (deflated 1%)
  adding: results/outdoor/Transweather/1928_0.9_0.08.jpg (deflated 0%)
  adding: results/outdoor/Transweather/0274_0.95_0.12.jpg (deflated 1%)
  adding: results/outdoor/Transweather/1949_0.8_0.2.jpg (deflated 0%)
  adding: results/outdoor/Transweather/0205_1_0.16.jpg (deflated 0%)
  adding: results/outdoor/T